# LightGBM basic


__Overview:__ This was initally forked from an existing kernel. It contains decent preprocessing prior to a JOIN. Specifically, taking the feature in the NEWS dataset that lists all relevent assets and expanding into asset rows. However, this can cause multiple entries for each date/asset in the NEWS data. Therefore, the features are averaged so that there is only one entry per date and asset. From there, the JOIN can be performed. 


Much of the asset feature engineering has been abstracted away at this point

Code for the kernel is commented out but included

This still has the aggregation needed for the JOIN

In [1]:
path = '/Users/jacob/Desktop/studies/misc/kaggle/two_sigma_news/scripts'
import resource
import sys
import pandas_datareader as web
 
sys.path.append(path)

from sigma_imports import *
from sigma_code import *
from assetExpand import *
tqdm.pandas()

In [55]:
# NEEDS TO BE ORGANIZED

import lightgbm
import warnings
warnings.filterwarnings(action='ignore')

import gc
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
import seaborn as sns
import matplotlib.pyplot as plt
# from kaggle.competitions import twosigmanews
pd.set_option('max_columns', 50)

In [2]:
# env = twosigmanews.make_env()
# DEMO DATA
market_train = pd.read_csv('../data/marketdata_sample.csv') 
news_train = pd.read_csv('../data/news_sample.csv')


## Create sample set

----------------------------

In [ ]:
# ASSETS WAS A CUT AND PASTE FROM KAGGLE - YOU CAN GRAB IT FROM THE TEXT FILE
# BUT IT'S A LOT
# LIST OF ASSETS PULLED FROM KERNEL; ASSETS = COPIED LIST IN KERNAL
with open('output/asset_kernel.txt', 'w') as file:
    file.write(str(assets))
file.close()    

In [18]:
with open('output/asset_kernel.txt', 'r') as file:
    assets = file.read()
    
# pretty messy w/o print  
print(assets[:20])   
file.close()
# consider using split() and regex to clean up - or stay lazy and paste it

['VNTV.N', 'AMGN.O',


In [ ]:
# WATCH THE TICKER EXTENSIONS
from pandas_datareader._utils import RemoteDataError

df_ = pd.DataFrame()
for i in assets:
#     print(i)
    
    i = i[:-2]
    print(i)
    try:
        vec = web.DataReader(i, 'yahoo', start='12/10/2006', end='2/15/2007')
        vec['asset'] = i
        vec['returns_close_raw'] = np.log(vec.Close/vec.Close.shift())
        vec['returns_open_raw'] = np.log(vec.Open/vec.Open.shift())
        vec['returns_open_raw10'] = np.log(vec.Open/vec.Open.shift(10))
        vec['returns_close_raw10'] = np.log(vec.Close/vec.Close.shift(10))
        vec['returns_open_raw10_next'] = np.log(vec.Open/vec.Open.shift(-10))
        df_ = pd.concat([df_, vec])
        
    except RemoteDataError:
        print('remote error')
    
    except KeyError:
        print('key error')
 

In [ ]:
df_.dropna(inplace=True) 
df_.sort_index(inplace=True)
# Rearrange cols
cols = ['asset', u'Open', u'Close',
u'Volume', 'returns_close_raw','returns_open_raw',
'returns_close_raw10','returns_open_raw10',
        'returns_open_raw10_next']
df = df_[cols]
# df.to_csv('output/sigma_data.csv')

In [25]:
sim_data = pd.read_csv('output/sigma_data.csv')
sim_data.rename(columns={'asset': 'assetTicker'}, inplace=True)

In [24]:
# GOOD
# sim_data.isnull().sum()

In [27]:
# NEED TO DO A LITTLE PREPROC FOR NEWS
pipe = AssetExpand()
news = pipe.fit_transform(news_train)

In [37]:
print(news.shape)
news.isnull().sum()
df = news[news.time.notnull()]
print(df_news.shape)

(282, 35)
(100, 35)


In [38]:

def ticker_ex():
    """
    quick way to separate the extensions - not ready.. paste to use
    """
    ext_vec = []
    asst_vec = []
    for i in df.assetCode:
        try:
            asst_vec.append(i.split('.')[0])
            # can do list comprehension for the first term
            ext_vec.append(i.split('.')[1])
        except IndexError:
            # some extension don't exist
            ext_vec.append(np.nan)
    df['ext'] = ext_vec
    df['assetTicker'] = asst_vec

ticker_ex()

In [41]:
def timestamp_conv(x):
    
    term = x.split()[0]
    return dt.datetime.strptime(term, '%Y-%m-%d')
  
    
df['Date'] = df.time.apply(timestamp_conv)
df.drop(['time'], axis=1, inplace=True)


In [46]:
# THIS IGNORES DATE - O.W. IT'S ALL NULLS
df_join = df.merge(sim_data, how='left', on=['assetTicker'])

In [49]:
df_join.isnull().sum()
df_sample = df_join[df_join.Volume.notnull()]
print(df_sample.shape)

(288, 46)


__This can be used to test transformations and such__

In [51]:
df_sample.head(3)

,assetCode,sourceTimestamp,firstCreated,sourceId,headline,urgency,takeSequence,provider,subjects,audiences,bodySize,companyCount,headlineTag,marketCommentary,sentenceCount,wordCount,assetName,firstMentionSentence,relevance,sentimentClass,sentimentNegative,sentimentNeutral,sentimentPositive,sentimentWordCount,noveltyCount12H,noveltyCount24H,noveltyCount3D,noveltyCount5D,noveltyCount7D,volumeCounts12H,volumeCounts24H,volumeCounts3D,volumeCounts5D,volumeCounts7D,ext,assetTicker,Date_x,Date_y,Open,Close,Volume,returns_close_raw,returns_open_raw,returns_close_raw10,returns_open_raw10,returns_open_raw10_next
5,WMT.N,2007-01-01 12:08:37+00:00,2007-01-01 12:08:37+00:00,23768af19dc69992,PRESS DIGEST - New York Times - Jan 1,3.0,1.0,RTRS,"{'PUB', 'BUS', 'INS', 'CA', 'ENT', 'US', 'FIN'...","{'M', 'PMF', 'DNP', 'PTD', 'T', 'E', 'D', 'PSC...",1776.0,6.0,PRESS DIGEST,False,14.0,325.0,Sirius XM Radio Inc,0.0,0.149071,-1.0,0.476228,0.336996,0.186776,325.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,N,WMT,2007-01-01,2006-12-26,44.799999,46.110001,10493300.0,0.012439,-0.015504,0.002388,-0.036168,-0.053668
6,WMT.N,2007-01-01 12:08:37+00:00,2007-01-01 12:08:37+00:00,23768af19dc69992,PRESS DIGEST - New York Times - Jan 1,3.0,1.0,RTRS,"{'PUB', 'BUS', 'INS', 'CA', 'ENT', 'US', 'FIN'...","{'M', 'PMF', 'DNP', 'PTD', 'T', 'E', 'D', 'PSC...",1776.0,6.0,PRESS DIGEST,False,14.0,325.0,Sirius XM Radio Inc,0.0,0.149071,-1.0,0.476228,0.336996,0.186776,325.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,N,WMT,2007-01-01,2006-12-27,45.750000,46.160000,12124100.0,0.001084,0.020984,0.011110,-0.003709,-0.040061
7,WMT.N,2007-01-01 12:08:37+00:00,2007-01-01 12:08:37+00:00,23768af19dc69992,PRESS DIGEST - New York Times - Jan 1,3.0,1.0,RTRS,"{'PUB', 'BUS', 'INS', 'CA', 'ENT', 'US', 'FIN'...","{'M', 'PMF', 'DNP', 'PTD', 'T', 'E', 'D', 'PSC...",1776.0,6.0,PRESS DIGEST,False,14.0,325.0,Sirius XM Radio Inc,0.0,0.149071,-1.0,0.476228,0.336996,0.186776,325.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,N,WMT,2007-01-01,2006-12-28,45.810001,46.009998,7100500.0,-0.003255,0.001311,0.002394,-0.001963,-0.045239


Let's remove data before 2009 (optional):

In [59]:
# start = datetime(2009, 1, 1, 0, 0, 0).date()
# market_train = market_train.loc[market_train['time'].dt.date >= start].reset_index(drop=True)
# news_train = news_train.loc[news_train['time'].dt.date >= start].reset_index(drop=True)

In [173]:
market_train.head(3)

,time,assetCode,assetName,universe,volume,close,open,returnsClosePrevRaw1,returnsOpenPrevRaw1,returnsClosePrevMktres1,returnsOpenPrevMktres1,returnsClosePrevRaw10,returnsOpenPrevRaw10,returnsClosePrevMktres10,returnsOpenPrevMktres10,returnsOpenNextMktres10
0,2007-02-01 22:00:00+00:00,A.N,Agilent Technologies Inc,1.0,2606900.0,32.19,32.17,0.005938,0.005312,NaN,NaN,-0.001860,0.000622,NaN,NaN,0.034672
1,2007-02-01 22:00:00+00:00,AAI.N,AirTran Holdings Inc,0.0,2051600.0,11.12,11.08,0.004517,-0.007168,NaN,NaN,-0.078708,-0.088066,NaN,NaN,0.027803
2,2007-02-01 22:00:00+00:00,AAP.N,Advance Auto Parts Inc,1.0,1164800.0,37.51,37.99,-0.011594,0.025648,NaN,NaN,0.014332,0.045405,NaN,NaN,0.024433


In [174]:
news_train.head(3)

,time,sourceId,urgency,takeSequence,provider,bodySize,companyCount,headlineTag,marketCommentary,sentenceCount,wordCount,assetCodes,firstMentionSentence,relevance,sentimentClass,sentimentNegative,sentimentNeutral,sentimentPositive,sentimentWordCount,noveltyCount12H,noveltyCount24H,noveltyCount3D,noveltyCount5D,noveltyCount7D,volumeCounts12H,volumeCounts24H,volumeCounts3D,volumeCounts5D,volumeCounts7D
0,2007-01-01 04:29:32+00:00,0,3,1,0,1438,1,-1,False,11,275,"{'PTR.N', '0857.F', '0857.DE', '0857.HK'}",6,0.235702,-1,0.500739,0.419327,0.079934,73,0,0,0,0,0,0,0,3,6,7
1,2007-01-01 07:03:35+00:00,1,3,1,0,4413,1,0,False,55,907,{'STA.N'},8,0.447214,-1,0.600082,0.345853,0.054064,62,1,1,1,1,1,1,1,3,3,3
2,2007-01-01 11:29:56+00:00,2,3,1,0,2108,2,1,False,15,388,"{'WMT.N', 'WMT.DE'}",14,0.377964,-1,0.450049,0.295671,0.254280,67,0,0,0,0,0,0,0,5,11,17


In [379]:
# Factorize is simply the equivalent to R encoding
# column_1, uniques = pd.factorize(news_train['sourceId'])


In [63]:
# DEFINITELY DON'T AGREE WITH THESE DROPS
drop_list = [
        'audiences', 'subjects', 'assetName',
        'headline', 'firstCreated', 'sourceTimestamp',
    ]

news_train.drop(drop_list, axis=1, inplace=True)
    

In [64]:
for col in ['headlineTag', 'provider', 'sourceId']:
        news_train[col], uniques = pd.factorize(news_train[col])
        del uniques

In [182]:
d_type_usage(news_train)

Average memory usage for float columns: 0.00 MB
Average memory usage for int columns: 0.00 MB
Average memory usage for object columns: 0.01 MB


In [190]:
# news_train.dtypes

In [65]:
news_train.head(3)

,time,sourceId,urgency,takeSequence,provider,bodySize,companyCount,headlineTag,marketCommentary,sentenceCount,wordCount,assetCodes,firstMentionSentence,relevance,sentimentClass,sentimentNegative,sentimentNeutral,sentimentPositive,sentimentWordCount,noveltyCount12H,noveltyCount24H,noveltyCount3D,noveltyCount5D,noveltyCount7D,volumeCounts12H,volumeCounts24H,volumeCounts3D,volumeCounts5D,volumeCounts7D
0,2007-01-01 04:29:32+00:00,0,3,1,0,1438,1,-1,False,11,275,"{'PTR.N', '0857.F', '0857.DE', '0857.HK'}",6,0.235702,-1,0.500739,0.419327,0.079934,73,0,0,0,0,0,0,0,3,6,7
1,2007-01-01 07:03:35+00:00,1,3,1,0,4413,1,0,False,55,907,{'STA.N'},8,0.447214,-1,0.600082,0.345853,0.054064,62,1,1,1,1,1,1,1,3,3,3
2,2007-01-01 11:29:56+00:00,2,3,1,0,2108,2,1,False,15,388,"{'WMT.N', 'WMT.DE'}",14,0.377964,-1,0.450049,0.295671,0.254280,67,0,0,0,0,0,0,0,5,11,17


<h2>3. Unstacking news</h2>

Assets are actually a list of codes in the news frame, but we need to merge with market data which has individual asset codes. Therefore, we are going to unstack each asset code and save the original index with the following function. This is probably not the best way of doing that, but it is simple:

In [390]:
# def unstack_asset_codes(news_train):
#     codes = []
#     indexes = []
#     for i, values in news_train['assetCodes'].iteritems():
#         explode = values.split(", ")
#         codes.extend(explode)
#         repeat_index = [int(i)]*len(explode)
#         indexes.extend(repeat_index)
#     index_df = pd.DataFrame({'news_index': indexes, 'assetCode': codes})
#     del codes, indexes
#     gc.collect()
#     return index_df

# index_df = unstack_asset_codes(news_train)
# index_df.head()

Now we can merge the news on this frame:

In [391]:
# def merge_news_on_index(news_train, index_df):
#     news_train['news_index'] = news_train.index.copy()

#     # Merge news on unstacked assets
#     news_unstack = index_df.merge(news_train, how='left', on='news_index')
#     news_unstack.drop(['news_index', 'assetCodes'], axis=1, inplace=True)
#     return news_unstack

# news_unstack = merge_news_on_index(news_train, index_df)
# del news_train, index_df
# gc.collect()
# news_unstack.head(3)

In [154]:
df_2 = df_news.copy()


In [155]:
df_2['Date'] = pd.to_datetime(df_2.time).dt.date  # Add date column
 

In [165]:
df_2.assetCode = df_2.assetCode.apply(lambda x: x.strip().split('.')[0])

In [167]:
data.head()

,Date,asset,Open,Close,Volume,returns_close_raw,returns_open_raw,returns_close_raw10,returns_open_raw10
0,2006-12-25,2330,736.000000,713.000000,415000.0,-0.110118,-0.089612,-0.324175,-0.345746
1,2006-12-26,VIA,39.750000,40.389999,91200.0,0.017735,0.009606,0.055747,0.053480
2,2006-12-26,RYAAY,41.092308,41.241024,165500.0,0.011506,0.007893,0.040734,0.043879
3,2006-12-26,MSFT,29.530001,29.990000,37098300.0,0.011739,-0.010108,0.015119,0.011581
4,2006-12-26,WMT,44.799999,46.110001,10493300.0,0.012439,-0.015504,0.002388,-0.036168


In [168]:
df_2.head()

,assetCode,time,sourceId,urgency,takeSequence,provider,bodySize,companyCount,headlineTag,marketCommentary,sentenceCount,wordCount,firstMentionSentence,relevance,sentimentClass,sentimentNegative,sentimentNeutral,sentimentPositive,sentimentWordCount,noveltyCount12H,noveltyCount24H,noveltyCount3D,noveltyCount5D,noveltyCount7D,volumeCounts12H,volumeCounts24H,volumeCounts3D,volumeCounts5D,volumeCounts7D,Date
0,PTR,2007-01-01 04:29:32+00:00,0,3,1,0,1438,1,-1,False,11,275,6,0.235702,-1,0.500739,0.419327,0.079934,73,0,0,0,0,0,0,0,3,6,7,2007-01-01
1,0857,2007-01-01 04:29:32+00:00,0,3,1,0,1438,1,-1,False,11,275,6,0.235702,-1,0.500739,0.419327,0.079934,73,0,0,0,0,0,0,0,3,6,7,2007-01-01
2,0857,2007-01-01 04:29:32+00:00,0,3,1,0,1438,1,-1,False,11,275,6,0.235702,-1,0.500739,0.419327,0.079934,73,0,0,0,0,0,0,0,3,6,7,2007-01-01
3,0857,2007-01-01 04:29:32+00:00,0,3,1,0,1438,1,-1,False,11,275,6,0.235702,-1,0.500739,0.419327,0.079934,73,0,0,0,0,0,0,0,3,6,7,2007-01-01
4,STA,2007-01-01 07:03:35+00:00,1,3,1,0,4413,1,0,False,55,907,8,0.447214,-1,0.600082,0.345853,0.054064,62,1,1,1,1,1,1,1,3,3,3,2007-01-01


In [169]:
   
aggregations = ['mean']
gp = df_2.groupby(['assetCode', 'Date']).agg(aggregations)
gp.columns = pd.Index(["{}_{}".format(e[0], e[1]) for e in gp.columns.tolist()])
gp.reset_index(inplace=True)
# Set datatype to float32
float_cols = {c: 'float32' for c in gp.columns if c not in ['assetCode', 'Date']}
# return gp.astype(float_cols)


In [170]:
df_agg = gp.astype(float_cols)


In [171]:
# float_cols
df_agg.head()


,assetCode,Date,sourceId_mean,urgency_mean,takeSequence_mean,provider_mean,bodySize_mean,companyCount_mean,headlineTag_mean,marketCommentary_mean,sentenceCount_mean,wordCount_mean,firstMentionSentence_mean,relevance_mean,sentimentClass_mean,sentimentNegative_mean,sentimentNeutral_mean,sentimentPositive_mean,sentimentWordCount_mean,noveltyCount12H_mean,noveltyCount24H_mean,noveltyCount3D_mean,noveltyCount5D_mean,noveltyCount7D_mean,volumeCounts12H_mean,volumeCounts24H_mean,volumeCounts3D_mean,volumeCounts5D_mean,volumeCounts7D_mean
0,0005,2007-01-02,42.500000,3.0,1.500000,0.0,959.500000,4.000000,5.0,0.000000,4.500000,152.000000,4.000000,0.408114,0.000000,0.021287,0.812900,0.165814,87.000000,0.500000,0.500000,0.500000,2.500000,2.500000,0.500000,0.500000,0.500000,4.500000,8.500000
1,015760,2007-01-02,34.428570,3.0,1.571429,0.0,3275.000000,4.000000,-1.0,0.571429,49.142857,636.428589,2.142857,0.640754,0.428571,0.210839,0.182847,0.606314,162.142853,1.571429,1.571429,1.571429,1.571429,1.571429,3.714286,3.714286,3.714286,7.714286,7.714286
2,034220,2007-01-02,49.000000,3.0,1.666667,0.0,4093.000000,5.000000,-1.0,0.333333,50.333332,760.666687,1.000000,1.000000,0.333333,0.280354,0.149233,0.570413,310.333344,0.333333,0.333333,0.333333,0.333333,0.333333,1.000000,1.000000,1.000000,1.000000,4.666667
3,053000,2007-01-02,34.000000,3.0,1.666667,0.0,4128.666504,5.666667,-1.0,0.666667,70.333336,820.333313,21.333334,0.079448,-0.333333,0.405031,0.136259,0.458710,70.666664,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,2.000000,2.000000,2.000000,4.000000
4,060000,2007-01-02,37.799999,3.0,1.400000,0.0,2983.000000,3.800000,0.0,0.400000,46.000000,585.599976,13.400000,0.352389,-0.200000,0.496662,0.138379,0.364959,79.199997,0.800000,0.800000,0.800000,0.800000,1.600000,3.400000,3.400000,3.400000,5.400000,10.200000


<h2>4. Group by date and asset</h2>

There can be many News for a single date and asset, so we need to group this data. I'll be using a simple mean, but you can use more intelligent features.

In [398]:
# def group_news(news_frame):
#     news_frame['date'] = news_frame.time.dt.date  # Add date column
    
#     aggregations = ['mean']
#     gp = news_frame.groupby(['assetCode', 'date']).agg(aggregations)
#     gp.columns = pd.Index(["{}_{}".format(e[0], e[1]) for e in gp.columns.tolist()])
#     gp.reset_index(inplace=True)
#     # Set datatype to float32
#     float_cols = {c: 'float32' for c in gp.columns if c not in ['assetCode', 'date']}
#     return gp.astype(float_cols)

# news_agg = group_news(df_news)
# # del news_unstack; gc.collect()
# news_agg.head(3)

<h2>5. Merge on Market data</h2>

### The dates do not match --> merge not possible. Need to use the kernel

The final preprocessing step is to merge news data with market data:

In [399]:
# market_train['date'] = market_train.time.dt.date

In [45]:
market_train['date'] = pd.to_datetime(market_train.time).dt.date  # Add date column
 

In [172]:
market_train.tail(50)


,time,assetCode,assetName,universe,volume,close,open,returnsClosePrevRaw1,returnsOpenPrevRaw1,returnsClosePrevMktres1,returnsOpenPrevMktres1,returnsClosePrevRaw10,returnsOpenPrevRaw10,returnsClosePrevMktres10,returnsOpenPrevMktres10,returnsOpenNextMktres10
50,2007-02-01 22:00:00+00:00,AIN.N,Albany International Corp,0.0,117700.0,34.29,33.94,0.010312,0.002955,NaN,NaN,0.048945,0.039510,NaN,NaN,-0.018636
51,2007-02-01 22:00:00+00:00,AINV.O,Apollo Investment Corp,1.0,961700.0,22.32,22.23,0.005405,0.003612,NaN,NaN,0.018248,0.005882,NaN,NaN,0.038204
52,2007-02-01 22:00:00+00:00,AIR.N,AAR Corp,0.0,967200.0,29.36,29.76,-0.014434,0.051590,NaN,NaN,0.033439,0.040196,NaN,NaN,0.040983
53,2007-02-01 22:00:00+00:00,AIT.N,Applied Industrial Technologies Inc,0.0,274400.0,25.10,24.56,0.023654,0.014876,NaN,NaN,0.038907,-0.004055,NaN,NaN,-0.024102
54,2007-02-01 22:00:00+00:00,AIV.N,Apartment Investment and Management Co,1.0,858000.0,62.59,62.63,-0.000639,0.008860,NaN,NaN,0.039874,0.049079,NaN,NaN,-0.031624
55,2007-02-01 22:00:00+00:00,AIZ.N,Assurant Inc,1.0,1020600.0,56.36,55.57,0.014034,-0.002334,NaN,NaN,-0.007746,-0.020793,NaN,NaN,-0.039343
56,2007-02-01 22:00:00+00:00,AJG.N,Arthur J Gallagher & Co,1.0,990800.0,28.77,28.75,0.003488,-0.014398,NaN,NaN,0.006296,0.001393,NaN,NaN,0.001590
57,2007-02-01 22:00:00+00:00,AKAM.O,Akamai Technologies Inc,1.0,2801000.0,55.51,56.61,-0.011926,0.016520,NaN,NaN,0.050530,0.064698,NaN,NaN,0.013804
58,2007-02-01 22:00:00+00:00,AKR.N,Acadia Realty Trust,0.0,251000.0,26.04,25.79,0.013624,0.006243,NaN,NaN,0.025197,0.013758,NaN,NaN,0.054497
59,2007-02-01 22:00:00+00:00,AKS.N,AK Steel Holding Corp,1.0,3464400.0,21.69,21.31,0.030894,0.009953,NaN,NaN,0.309783,0.266191,NaN,NaN,0.042568


In [54]:
df_agg.head()

,assetCode,date,sourceId_mean,urgency_mean,takeSequence_mean,provider_mean,bodySize_mean,companyCount_mean,headlineTag_mean,marketCommentary_mean,sentenceCount_mean,wordCount_mean,firstMentionSentence_mean,relevance_mean,sentimentClass_mean,sentimentNegative_mean,sentimentNeutral_mean,sentimentPositive_mean,sentimentWordCount_mean,noveltyCount12H_mean,noveltyCount24H_mean,noveltyCount3D_mean,noveltyCount5D_mean,noveltyCount7D_mean,volumeCounts12H_mean,volumeCounts24H_mean,volumeCounts3D_mean,volumeCounts5D_mean,volumeCounts7D_mean
0,0005.HK,2007-01-02,42.5,3.0,1.500000,0.0,959.500000,4.000000,5.0,0.000000,4.500000,152.000000,4.000000,0.408114,0.000000,0.021287,0.812900,0.165814,87.000000,0.500000,0.500000,0.500000,2.500000,2.500000,0.5,0.5,0.5,4.5,8.500000
1,034220.KS,2007-01-02,49.0,3.0,1.666667,0.0,4093.000000,5.000000,-1.0,0.333333,50.333332,760.666687,1.000000,1.000000,0.333333,0.280354,0.149233,0.570413,310.333344,0.333333,0.333333,0.333333,0.333333,0.333333,1.0,1.0,1.0,1.0,4.666667
2,053000.KS,2007-01-02,34.0,3.0,1.666667,0.0,4128.666504,5.666667,-1.0,0.666667,70.333336,820.333313,21.333334,0.079448,-0.333333,0.405031,0.136259,0.458710,70.666664,1.000000,1.000000,1.000000,1.000000,1.000000,2.0,2.0,2.0,2.0,4.000000
3,0857.DE,2007-01-01,0.0,3.0,1.000000,0.0,1438.000000,1.000000,-1.0,0.000000,11.000000,275.000000,6.000000,0.235702,-1.000000,0.500739,0.419327,0.079934,73.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,3.0,6.0,7.000000
4,0857.F,2007-01-01,0.0,3.0,1.000000,0.0,1438.000000,1.000000,-1.0,0.000000,11.000000,275.000000,6.000000,0.235702,-1.000000,0.500739,0.419327,0.079934,73.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,3.0,6.0,7.000000


In [53]:
# df = market_train.merge(df_agg, how='left', on=['assetCode', 'date'])
df = market_train.merge(df_agg, how='left', on=['assetCode'])
# del market_train, news_agg
# gc.collect()
df.head(50)

,time,assetCode,assetName,universe,volume,close,open,returnsClosePrevRaw1,returnsOpenPrevRaw1,returnsClosePrevMktres1,returnsOpenPrevMktres1,returnsClosePrevRaw10,returnsOpenPrevRaw10,returnsClosePrevMktres10,returnsOpenPrevMktres10,returnsOpenNextMktres10,date_x,date_y,sourceId_mean,urgency_mean,takeSequence_mean,provider_mean,bodySize_mean,companyCount_mean,headlineTag_mean,marketCommentary_mean,sentenceCount_mean,wordCount_mean,firstMentionSentence_mean,relevance_mean,sentimentClass_mean,sentimentNegative_mean,sentimentNeutral_mean,sentimentPositive_mean,sentimentWordCount_mean,noveltyCount12H_mean,noveltyCount24H_mean,noveltyCount3D_mean,noveltyCount5D_mean,noveltyCount7D_mean,volumeCounts12H_mean,volumeCounts24H_mean,volumeCounts3D_mean,volumeCounts5D_mean,volumeCounts7D_mean
0,2007-02-01 22:00:00+00:00,A.N,Agilent Technologies Inc,1.0,2606900.0,32.19,32.17,0.005938,0.005312,NaN,NaN,-0.001860,0.000622,NaN,NaN,0.034672,2007-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2007-02-01 22:00:00+00:00,AAI.N,AirTran Holdings Inc,0.0,2051600.0,11.12,11.08,0.004517,-0.007168,NaN,NaN,-0.078708,-0.088066,NaN,NaN,0.027803,2007-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2007-02-01 22:00:00+00:00,AAP.N,Advance Auto Parts Inc,1.0,1164800.0,37.51,37.99,-0.011594,0.025648,NaN,NaN,0.014332,0.045405,NaN,NaN,0.024433,2007-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2007-02-01 22:00:00+00:00,AAPL.O,Apple Inc,1.0,23747329.0,84.74,86.23,-0.011548,0.016324,NaN,NaN,-0.048613,-0.037182,NaN,NaN,-0.007425,2007-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2007-02-01 22:00:00+00:00,ABB.N,ABB Ltd,1.0,1208600.0,18.02,18.01,0.011791,0.025043,NaN,NaN,0.012929,0.020397,NaN,NaN,-0.017994,2007-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2007-02-01 22:00:00+00:00,ABC.N,AmerisourceBergen Corp,1.0,1657300.0,52.37,52.40,-0.000191,0.008468,NaN,NaN,0.089000,0.077746,NaN,NaN,0.058680,2007-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2007-02-01 22:00:00+00:00,ABD.N,ACCO Brands Corp,0.0,1186200.0,23.63,24.13,-0.020721,-0.007404,NaN,NaN,0.005104,0.026809,NaN,NaN,-0.044285,2007-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2007-02-01 22:00:00+00:00,ABM.N,ABM Industries Inc,0.0,301200.0,26.19,25.90,0.013545,0.014890,NaN,NaN,0.068980,0.047311,NaN,NaN,0.016578,2007-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2007-02-01 22:00:00+00:00,ABT.N,Abbott Laboratories,1.0,5692300.0,52.87,52.50,-0.002453,-0.004739,NaN,NaN,0.001515,-0.004928,NaN,NaN,0.009861,2007-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2007-02-01 22:00:00+00:00,ABV.N,Companhia de Bebidas das Americas Ambev,1.0,401800.0,52.46,52.10,0.014112,0.024784,NaN,NaN,0.042321,0.027411,NaN,NaN,0.012917,2007-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<h2>6. Train GBM model </h2>

This competition has a custom metric (check the evaluation tab). The following function returns the custom metric from the probability of each example being positive (or 1).

In [ ]:
def custom_metric(date, pred_proba, num_target, universe):
    y = pred_proba*2 - 1
    r = num_target.clip(-1,1) # get rid of outliers
    x = y * r * universe
    result = pd.DataFrame({'day' : date, 'x' : x})
    x_t = result.groupby('day').sum().values
    return np.mean(x_t) / np.std(x_t)

Drop columns that we don't need and set type to float32:

In [ ]:
date = df.date
num_target = df.returnsOpenNextMktres10.astype('float32')
bin_target = (df.returnsOpenNextMktres10 >= 0).astype('int8')
universe = df.universe.astype('int8')
# Drop columns that are not features
df.drop(['returnsOpenNextMktres10', 'date', 'universe', 'assetCode', 'assetName', 'time'], 
        axis=1, inplace=True)
df = df.astype('float32')  # Set all remaining columns to float32 datatype
gc.collect()

We can use the last 10% of data (aprox one year) to validate our model. You have to be careful when using different validation techniques like KFold since the time is important here.

In [ ]:
train_index, test_index = train_test_split(df.index.values, test_size=0.1, shuffle=False)

In [ ]:
def evaluate_model(df, target, train_index, test_index, params):
    params['n_jobs'] = 2  # Use 2 cores/threads
    #model = XGBClassifier(**params)
    model = LGBMClassifier(**params)
    model.fit(df.iloc[train_index], target.iloc[train_index])
    return log_loss(target.iloc[test_index], model.predict_proba(df.iloc[test_index]))

We can use a simple random search to find some hyperparameters:

In [ ]:
param_grid = {
    'learning_rate': [0.15, 0.1, 0.05, 0.02, 0.01],
    'num_leaves': [i for i in range(12, 90, 6)],
    'n_estimators': [50, 200, 400, 600, 800],
    'min_child_samples': [i for i in range(10, 100, 10)],
    'colsample_bytree': [0.8, 0.9, 0.95, 1],
    'subsample': [0.8, 0.9, 0.95, 1],
    'reg_alpha': [0.1, 0.2, 0.4, 0.6, 0.8],
    'reg_lambda': [0.1, 0.2, 0.4, 0.6, 0.8],
}

best_eval_score = 0
for i in range(100):  # Hundred runs
    params = {k: np.random.choice(v) for k, v in param_grid.items()}
    score = evaluate_model(df, bin_target, train_index, test_index, params)
    if score < best_eval_score or best_eval_score == 0:
        best_eval_score = score
        best_params = params
print("Best evaluation logloss", best_eval_score)

<h2>7. Make predictions and submit</h2>

In order to make predictions for the test set we must use the *env.predict* function and for our final submission *env.write_submission_file*. Otherwise, the script will fail in the second stage, when Kaggle will replace the "fake" data for 2019 market data and run our scripts again.

In [ ]:
# Train model with full data
clf = LGBMClassifier(**best_params)
clf.fit(df, bin_target)

In [ ]:
def write_submission(model, env):
    days = env.get_prediction_days()
    for (market_obs_df, news_obs_df, predictions_template_df) in days:
        news_obs_df = preprocess_news(news_obs_df)
        # Unstack news
        index_df = unstack_asset_codes(news_obs_df)
        news_unstack = merge_news_on_index(news_obs_df, index_df)
        # Group and and get aggregations (mean)
        news_obs_agg = group_news(news_unstack)

        # Join market and news frames
        market_obs_df['date'] = market_obs_df.time.dt.date
        obs_df = market_obs_df.merge(news_obs_agg, how='left', on=['assetCode', 'date'])
        del market_obs_df, news_obs_agg, news_obs_df, news_unstack, index_df
        gc.collect()
        obs_df = obs_df[obs_df.assetCode.isin(predictions_template_df.assetCode)]
        
        # Drop cols that are not features
        feats = [c for c in obs_df.columns if c not in ['date', 'assetCode', 'assetName', 'time']]

        preds = model.predict_proba(obs_df[feats])[:, 1] * 2 - 1
        sub = pd.DataFrame({'assetCode': obs_df['assetCode'], 'confidence': preds})
        predictions_template_df = predictions_template_df.merge(sub, how='left').drop(
            'confidenceValue', axis=1).fillna(0).rename(columns={'confidence':'confidenceValue'})
        
        env.predict(predictions_template_df)
        del obs_df, predictions_template_df, preds, sub
        gc.collect()
    env.write_submission_file()
    
write_submission(clf, env)

<h2>8. Feature importance</h2>

We can use Seaborn to plot the feature importance (with gain or number of splits criteria):

In [ ]:
feat_importance = pd.DataFrame()
feat_importance["feature"] = df.columns
feat_importance["gain"] = clf.booster_.feature_importance(importance_type='gain')
feat_importance.sort_values(by='gain', ascending=False, inplace=True)
plt.figure(figsize=(8,10))
ax = sns.barplot(y="feature", x="gain", data=feat_importance)

TODO

* Use custom metric
* Improve aggregations

Thanks for reading! Please upvote if you find usefull.

### Notes